<a href="https://colab.research.google.com/github/w-okada/voice-changer/blob/dev/VoiceChangerDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MMVCプレイヤー（普通版）
---

このノートはColab上でMMVCのボイチェンを行うノートです。

正式版はローカルPCのDocker上で動かすアプリケーションです。

正式版は、多くの場合より少ないタイムラグで滑らかに音声を変換できます。

詳細な使用方法はこちらの[リポジトリ](https://github.com/w-okada/voice-changer)からご確認ください。


# GPUを確認
GPUを用いたほうが高速に処理が行えます。

下記のコマンドでGPUが確認できない場合は、上のメニューから

「ランタイム」→「ランタイムの変更」→「ハードウェア アクセラレータ」

でGPUを選択してください。

In [1]:
# (1) GPUの確認
!nvidia-smi

Thu Jan 12 09:17:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 使用するモデルとコンフィグファイルの指定

使用するトレーニング済みのモデルと、トレーニングで使用したコンフィグファイルのパスを指定してください。

多くの場合はGoogle Driveに格納されているファイルを使用すると思います。その場合は、下の(2-2)のセルを実行してドライブをマウントしてください

In [19]:
# (2-1) 使用するモデルとコンフィグファイルの指定
if "MODEL" in locals():
  del MODEL
if "ONNX" in locals():
  del ONNX

CONFIG="/content/drive/MyDrive/VoiceChanger/config.json"
#MODEL="/content/drive/MyDrive/VoiceChanger/G_326000.pth"
ONNX="/content/drive/MyDrive/VoiceChanger/G_326000.onnx"

In [3]:
# (2-2) Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# リポジトリのクローン
リポジトリをクローンします

In [4]:
# (3) リポジトリのクローン
!git clone --depth 1 https://github.com/w-okada/voice-changer.git  -b v.1.3.2
%cd voice-changer/server
!git clone --depth 1 https://github.com/isletennos/MMVC_Trainer.git -b v1.3.2.2
!cd MMVC_Trainer/monotonic_align/ && python setup.py build_ext --inplace && cd -

Cloning into 'voice-changer'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 143 (delta 21), reused 57 (delta 5), pack-reused 0
Receiving objects: 100% (143/143), 1.51 MiB | 2.50 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/voice-changer/server
Cloning into 'MMVC_Trainer'...
remote: Enumerating objects: 920, done.
remote: Counting objects: 100% (920/920), done.
remote: Compressing objects: 100% (830/830), done.
remote: Total 920 (delta 4), reused 893 (delta 1), pack-reused 0
Receiving objects: 100% (920/920), 53.04 MiB | 17.53 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Note: checking out 'f17c8c57d1ab7512633e6c57521f1eef6851bd0e'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to

# モジュールのインストール

必要なモジュールをインストールします。

In [5]:
# (5) 設定ファイルの確認
!apt-get install -y espeak libsndfile1-dev &> /dev/null
!pip install unidecode &> /dev/null
!pip install phonemizer &> /dev/null
!pip install retry &> /dev/null
!pip install python-socketio &> /dev/null
!pip install fastapi &> /dev/null
!pip install python-multipart &> /dev/null
!pip install uvicorn &> /dev/null
!pip install websockets &> /dev/null
!pip install pyOpenSSL &> /dev/null
!pip install onnxruntime-gpu &> /dev/null

# サーバの起動

サーバを起動します。(6-1)

サーバの起動状況を確認します。(6-2) 

このセルは繰り返し実行することになるのでCtrl+Retでセルを実行してください。

アクセスできるようになるまで、１～２分かかるようです。コーヒーでも飲みに行きましょう。

下記のようなテキストが表示されたら起動完了です。

**`INFO:root:Loaded checkpoint ...`**

```
    Phase name:__main__
    PHASE3:__main__
VoiceChanger Initialized (GPU_NUM:1, mps_enabled:False)
INFO:root:Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)
    PHASE1:__main__
Start MMVC SocketIO Server
    CONFIG:/content/drive/MyDrive/VoiceChanger/config.json, MODEL:/content/drive/MyDrive/VoiceChanger/G_326000.pth ONNX_MODEL:None
    Phase name:MMVCServerSIO
    PHASE3:MMVCServerSIO
INFO:root:Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)
```



In [20]:
# (6-1) サーバの起動
import random
PORT = 10000 + random.randint(1, 9999)
LOG_FILE = f"LOG_FILE_{PORT}"

if "MODEL" in locals() and "ONNX" in locals():
  model_param = f" -m {MODEL} -o {ONNX}"
elif "MODEL" in locals():
  model_param = f" -m {MODEL}"
elif "ONNX" in locals():
  model_param = f" -o {ONNX}"
else:
  model_param = f""

get_ipython().system_raw(f'python3 MMVCServerSIO.py -t MMVC -p {PORT} -c {CONFIG} {model_param} --colab True >{LOG_FILE} 2>&1 &')
#print(f"PORT:{PORT}, LOG_FILE:{LOG_FILE}")

python3 MMVCServerSIO.py -t MMVC -p 16400 -c /content/drive/MyDrive/VoiceChanger/config.json  -m /content/drive/MyDrive/VoiceChanger/G_326000.pth -o /content/drive/MyDrive/VoiceChanger/G_326000.onnx --colab True >LOG_FILE_16400 2>&1 &


In [13]:
# (6-2) サーバの起動確認 (Ctrl+Retで実行)
!tail -20 {LOG_FILE}

    Phase name:__main__
    PHASE3:__main__
VoiceChanger Initialized (GPU_NUM:1, mps_enabled:False)
INFO:root:Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)
    PHASE1:__main__
Start MMVC SocketIO Server
    CONFIG:/content/drive/MyDrive/VoiceChanger/config.json, MODEL:/content/drive/MyDrive/VoiceChanger/G_326000.pth ONNX_MODEL:None
    Phase name:MMVCServerSIO
    PHASE3:MMVCServerSIO
INFO:root:Loaded checkpoint '/content/drive/MyDrive/VoiceChanger/G_326000.pth' (iteration 1136)


# プロキシを起動
ウェブサーバへのアクセスをするためのプロキシを起動します。

表示されたURLをクリックして開くと別タブでアプリが開きます。

Colabなので、ロードにある程度時間がかかります(30秒くらい)。

In [14]:
# (7) プロキシを起動
from google.colab.output import eval_js
proxy = eval_js( "google.colab.kernel.proxyPort(" + str(PORT) + ")" )
print(f"{proxy}front/")

https://q27c1y5n1t-496ff2e9c6d22116-13348-colab.googleusercontent.com/front/
